In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

In [2]:
df_train = pd.read_csv(r'C:\Users\pauri\OneDrive\Escritorio\4) MÁSTER EN DATA SCIENCE, BIG DATA & BUSINESS ANALYTICS (UCM)\MÓDULO 8_MACHINE LEARNING\Clase 1 y 2\3. PRE_PROCESADO\titanic_kaggle\train.csv')

In [24]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
df_train['SibSp'].value_counts()

SibSp
0    608
1    209
2     28
4     18
3     16
8      7
5      5
Name: count, dtype: int64

In [30]:
# Porcentajes sobre el total
porcentajes_total = df_train.groupby('Pclass')['Survived'].value_counts() / len(df_train) * 100
porcentajes_total

Pclass  Survived
1       1           15.263749
        0            8.978676
2       0           10.886644
        1            9.764310
3       0           41.750842
        1           13.355780
Name: count, dtype: float64

In [33]:
# Porcentajes para cada clase
porcentajes_clase = df_train.groupby('Pclass')['Survived'].value_counts(normalize=True) *100
porcentajes_clase

Pclass  Survived
1       1           62.962963
        0           37.037037
2       0           52.717391
        1           47.282609
3       0           75.763747
        1           24.236253
Name: proportion, dtype: float64

In [34]:
# O formato tabla
porcentajes_clase_tabla = df_train.groupby('Pclass')['Survived'].value_counts(normalize=True).unstack()
porcentajes_clase_tabla

Survived,0,1
Pclass,,
1,0.370370,0.629630
2,0.527174,0.472826
3,0.757637,0.242363


In [5]:
df_train.shape

(891, 12)

In [6]:
len(df_train['Ticket'].unique())

681

### Análisis de variables

### 1- Detección de valores nulos (detección no es imputación)

Los datos perdidos son la ausencia de valores en ciertas observaciones de una variable. Los datos que faltan son un problema inevitable en la mayoría de las fuentes de datos y pueden tener un impacto significativo en las conclusiones que derivamos de los datos. 

¿Por qué faltan datos?
El origen de los datos que faltan puede variar. Estos son sólo algunos ejemplos:

El valor se olvidó, se perdió o no se almacenó correctamente.

El valor no existe.

El valor no puede conocerse o identificarse.

Por poner ejemplos de la vida real, una persona puede decidir no rellenar todos los campos de un formulario si no son obligatorios. Eso introduciría datos que faltan. A veces las personas no quieren revelar cierta información, por ejemplo, los ingresos, o no conocen las respuestas a las preguntas que se les hacen.

A veces el valor de una determinada variable no existe. Por ejemplo, en la variable «deuda total como porcentaje de los ingresos totales» (muy común en los datos financieros), si la persona no tiene ingresos, entonces el porcentaje total de 0 no existe, y por lo tanto será un valor omitido.

Es importante entender por qué faltan datos, en otras palabras, el mecanismo de los datos que faltan. Podemos procesar la información que falta de forma diferente en función de este mecanismo. Además, identificar la fuente de los datos que faltan nos permite tomar medidas para regular esa fuente y reducir la cantidad de datos que faltan a medida que avanza la recogida de datos.

Existen tres mecanismos que provocan la falta de datos. Dos de ellos implican la pérdida de datos de forma aleatoria y el tercero implica una pérdida sistemática de datos.

-MCAR:
Si la probabilidad de que falte un valor es la misma para todas las observaciones, la variable falta completamente al azar (MCAR). Los puntos de datos que faltan son un subconjunto aleatorio de las observaciones.

Si los datos son MCAR, descartar las observaciones con datos perdidos no sesgaría las inferencias realizadas.

-MAR:
Si la probabilidad de que falte una observación depende de la información disponible (es decir, de otras variables), entonces la observación falta al azar (MAR). Existe una relación entre la probabilidad de que falte un valor y los datos observados.

Por ejemplo, si es más probable que los hombres revelen su peso que las mujeres, el peso es MAR. La información sobre el peso faltará aleatoriamente para los hombres y las mujeres que no revelaron su peso, pero como los hombres son más propensos a revelarlo, habrá más valores que falten para las mujeres que para los hombres.

En este ejemplo, los puntos de datos que faltan ya no son un subconjunto aleatorio del total de observaciones.

Si decidimos proceder con la variable con valores perdidos (en este caso el peso), es posible que queramos incluir la variable género para controlar el sesgo en el peso de las observaciones perdidas.

- Datos perdidos no aleatorios, MNAR:
Si hay un mecanismo o una razón por la que faltan datos, entonces esos datos faltan de forma no aleatoria (MNAR). Por ejemplo, si las personas no rellenan una encuesta sobre depresión debido a su depresión, esos puntos de datos faltarían de forma no aleatoria. Los datos que faltan se deben a la depresión.

Dependiendo del mecanismo por el que se produzcan los valores perdidos, podemos elegir diferentes métodos de imputación de datos perdidos.

In [31]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### 2- Rellenado de valores nulos

### 3- Cardinalidad

### 4- Conversión a numérica

### 5- Normalización

### 6- Detección outliers (no eliminación)